# Data Cleaning and Transforming

## Extract data (import)

In [1]:
import pandas as pd
import numpy as np
import pycountry_convert as pc

In [2]:
raw_data = pd.read_csv('climate_change_data.csv')
raw_data.head()

,Date,Location,Country,Temperature,CO2 Emissions,Sea Level Rise,Precipitation,Humidity,Wind Speed
0,2000-01-01 00:00:00.000000000,New Williamtown,Latvia,10.688986,403.118903,0.717506,13.835237,23.631256,18.492026
1,2000-01-01 20:09:43.258325832,North Rachel,South Africa,13.814430,396.663499,1.205715,40.974084,43.982946,34.249300
2,2000-01-02 16:19:26.516651665,West Williamland,French Guiana,27.323718,451.553155,-0.160783,42.697931,96.652600,34.124261
3,2000-01-03 12:29:09.774977497,South David,Vietnam,12.309581,422.404983,-0.475931,5.193341,47.467938,8.554563
4,2000-01-04 08:38:53.033303330,New Scottburgh,Moldova,13.210885,410.472999,1.135757,78.695280,61.789672,8.001164


In [3]:
raw_data.describe(include='all')

,Date,Location,Country,Temperature,CO2 Emissions,Sea Level Rise,Precipitation,Humidity,Wind Speed
count,10000,10000,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
unique,10000,7764,243,NaN,NaN,NaN,NaN,NaN,NaN
top,2000-01-01 00:00:00.000000000,North David,Congo,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,12,94,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,14.936034,400.220469,-0.003152,49.881208,49.771302,25.082066
std,NaN,NaN,NaN,5.030616,49.696933,0.991349,28.862417,28.929320,14.466648
min,NaN,NaN,NaN,-3.803589,182.131220,-4.092155,0.010143,0.018998,0.001732
25%,NaN,NaN,NaN,11.577991,367.109330,-0.673809,24.497516,24.713250,12.539733
50%,NaN,NaN,NaN,14.981136,400.821324,0.002332,49.818967,49.678412,24.910787
75%,NaN,NaN,NaN,18.305826,433.307905,0.675723,74.524991,75.206390,37.670260


# Transform Data

### Adding continent column to dataset by mapping country codes using pycountry-convert library

In [4]:
# Dictionary for mapping continent code to name
continentDict = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe'
}
# Mapping country by continent
continentlist=[]
for i in raw_data['Country']:
    try:
        country_code=pc.country_name_to_country_alpha2(i, cn_name_format="default") # convert country name to iso_code
        cont_code=pc.country_alpha2_to_continent_code(country_code) # convert iso_code to continent code
        cont_name=continentDict[cont_code] # convert continent code to name
    except KeyError:
        cont_name=None # Handler if the country name is not identified
    continentlist.append(cont_name)

In [7]:
print("length of list equal to the data shape =",len(continentlist)==raw_data.shape[0])
continentlist[:5]

length of list equal to the data shape = True


['Europe', 'Africa', 'South America', 'Asia', 'Europe']

In [8]:
# add continent column to dataframe
raw_data['Continent']=continentlist
raw_data.head()

,Date,Location,Country,Temperature,CO2 Emissions,Sea Level Rise,Precipitation,Humidity,Wind Speed,Continent
0,2000-01-01 00:00:00.000000000,New Williamtown,Latvia,10.688986,403.118903,0.717506,13.835237,23.631256,18.492026,Europe
1,2000-01-01 20:09:43.258325832,North Rachel,South Africa,13.814430,396.663499,1.205715,40.974084,43.982946,34.249300,Africa
2,2000-01-02 16:19:26.516651665,West Williamland,French Guiana,27.323718,451.553155,-0.160783,42.697931,96.652600,34.124261,South America
3,2000-01-03 12:29:09.774977497,South David,Vietnam,12.309581,422.404983,-0.475931,5.193341,47.467938,8.554563,Asia
4,2000-01-04 08:38:53.033303330,New Scottburgh,Moldova,13.210885,410.472999,1.135757,78.695280,61.789672,8.001164,Europe


In [9]:
raw_data.groupby('Continent').count()

,Date,Location,Country,Temperature,CO2 Emissions,Sea Level Rise,Precipitation,Humidity,Wind Speed
Continent,,,,,,,,,
Africa,2195,2195,2195,2195,2195,2195,2195,2195,2195
Asia,2023,2023,2023,2023,2023,2023,2023,2023,2023
Australia,1000,1000,1000,1000,1000,1000,1000,1000,1000
Europe,1889,1889,1889,1889,1889,1889,1889,1889,1889
North America,1471,1471,1471,1471,1471,1471,1471,1471,1471
South America,594,594,594,594,594,594,594,594,594


### Handling country names that is not identified by the library

In [10]:
set(raw_data.loc[raw_data['Continent'].isna()]['Country'])

{'Antarctica (the territory South of 60 deg S)',
 'Bouvet Island (Bouvetoya)',
 'British Indian Ocean Territory (Chagos Archipelago)',
 "Cote d'Ivoire",
 'French Southern Territories',
 'Heard Island and McDonald Islands',
 'Holy See (Vatican City State)',
 'Korea',
 'Libyan Arab Jamahiriya',
 'Netherlands Antilles',
 'Palestinian Territory',
 'Pitcairn Islands',
 'Reunion',
 'Saint Barthelemy',
 'Saint Helena',
 'Slovakia (Slovak Republic)',
 'Svalbard & Jan Mayen Islands',
 'Timor-Leste',
 'United States Minor Outlying Islands',
 'Western Sahara'}

Most of the country names that is not identified are islands/not a country. Some country names that is indeed a country are `Korea`, `Libyan Arab Jamahiriya` → `Libya`, `Cote d'Ivoire`, `Slovakia (Slovak Republic)` → `Slovakia`, `Timor-Leste`. I will map them manually since there is only 6 countries.

### Mapping unidentified countries

In [11]:
raw_data.loc[raw_data['Country'].isin(['Korea','Timor-Leste','Palestinian Territory']), 'Continent']='Asia'
raw_data.loc[raw_data['Country'] == 'Slovakia (Slovak Republic)', 'Continent']='Europe'
raw_data.loc[raw_data['Country'].isin(["Cote d'Ivoire", 'Libyan Arab Jamahiriya']), 'Continent']='Africa'

In [13]:
set(raw_data.loc[raw_data['Continent'].isna()]['Country'])

{'Antarctica (the territory South of 60 deg S)',
 'Bouvet Island (Bouvetoya)',
 'British Indian Ocean Territory (Chagos Archipelago)',
 'French Southern Territories',
 'Heard Island and McDonald Islands',
 'Holy See (Vatican City State)',
 'Netherlands Antilles',
 'Pitcairn Islands',
 'Reunion',
 'Saint Barthelemy',
 'Saint Helena',
 'Svalbard & Jan Mayen Islands',
 'United States Minor Outlying Islands',
 'Western Sahara'}

In [14]:
raw_data.loc[raw_data['Continent'].isna()]

,Date,Location,Country,Temperature,CO2 Emissions,Sea Level Rise,Precipitation,Humidity,Wind Speed,Continent
5,2000-01-05 04:48:36.291629162,South Nathan,Saint Helena,6.229326,392.473317,1.122210,76.368331,48.973886,30.398908,None
22,2000-01-19 11:33:51.683168316,East Darlene,French Southern Territories,21.082149,421.304289,0.236963,95.566283,87.701528,34.050236,None
29,2000-01-25 08:41:54.491449144,Williambury,Reunion,8.940555,398.027810,-0.733600,41.271330,7.689676,43.446416,None
37,2000-02-01 01:59:40.558055805,West Jeffrey,British Indian Ocean Territory (Chagos Archipe...,14.895743,474.318087,0.030541,46.626748,2.646945,20.247608,None
48,2000-02-10 07:46:36.399639964,Reynoldsberg,Reunion,9.086153,382.468520,-0.679688,36.860851,25.256477,26.707505,None
...,...,...,...,...,...,...,...,...,...,...
9969,2022-12-05 19:08:22.250225024,Bakershire,Western Sahara,13.841512,392.264074,-1.194432,55.543819,62.605059,35.589569,None
9977,2022-12-12 12:26:08.316831616,Simmonsburgh,Reunion,8.040745,471.626365,0.867123,56.003999,95.338273,42.129883,None
9983,2022-12-17 13:24:27.866786688,North Stephen,British Indian Ocean Territory (Chagos Archipe...,15.763664,435.153922,0.763351,11.269627,43.034142,18.590588,None
9986,2022-12-20 01:53:37.641764096,Johnview,Western Sahara,9.878773,506.129390,-1.508617,57.910260,94.057981,11.053514,None


### Add day, month, year from date to data

In [27]:
raw_data['Date']=pd.to_datetime(raw_data['Date'])
raw_data['day']=raw_data['Date'].dt.strftime('%d')
raw_data['month']=raw_data['Date'].dt.strftime('%m')
raw_data['year']=raw_data['Date'].dt.strftime('%Y')
raw_data.head()['day']

0    01
1    01
2    02
3    03
4    04
Name: day, dtype: object

In [29]:
raw_data.rename(columns={'Date':'Timestamp'}, inplace=True)
raw_data['date']=pd.to_datetime(raw_data['Timestamp']).dt.date
raw_data.head()

,Timestamp,Location,Country,Temperature,CO2 Emissions,Sea Level Rise,Precipitation,Humidity,Wind Speed,Continent,day,month,year,date
0,2000-01-01 00:00:00.000000000,New Williamtown,Latvia,10.688986,403.118903,0.717506,13.835237,23.631256,18.492026,Europe,01,01,2000,2000-01-01
1,2000-01-01 20:09:43.258325832,North Rachel,South Africa,13.814430,396.663499,1.205715,40.974084,43.982946,34.249300,Africa,01,01,2000,2000-01-01
2,2000-01-02 16:19:26.516651665,West Williamland,French Guiana,27.323718,451.553155,-0.160783,42.697931,96.652600,34.124261,South America,02,01,2000,2000-01-02
3,2000-01-03 12:29:09.774977497,South David,Vietnam,12.309581,422.404983,-0.475931,5.193341,47.467938,8.554563,Asia,03,01,2000,2000-01-03
4,2000-01-04 08:38:53.033303330,New Scottburgh,Moldova,13.210885,410.472999,1.135757,78.695280,61.789672,8.001164,Europe,04,01,2000,2000-01-04


# Loading cleaned data to csv format

In [30]:
raw_data.to_csv('climate-cleaned.csv', index=False)